In [124]:
import numpy as np
import gzip
import time
import glob
from sklearn.neighbors import NearestNeighbors
to=time.time()

fullcode={b"HIS":b"0",
        b"ASP":b"1",
        b"LYS":b"2",
        b"TYR":b"3",
        b"GLU":b"4",
        b"CYS":b"5"}
code={b"H":b"0",
        b"A":b"1",
        b"L":b"2",
        b"T":b"3",
        b"G":b"4",
        b"C":b"5"}
elements = {
    b"D":1, b"H": 1, b"LI":3, b"C": 6, b"N": 7, b"O": 8, 
    b"F":9,
    b"NA": 11, b"MG": 12,  b"P": 15, b"S": 16, b"CL": 17, b"K": 19,
    b"CA": 20, b"MN": 25, b"FE": 26, b"CO":27, b"NI":28,
    b"CU": 29, b"ZN": 30,
    b"SE":34,
    b"MO":42, b"SN":50, 
    b"I":53, b"CS":55,
    b"W":74, b"PT":78
    }
elements={k.decode() :np.int32(v) for k,v in elements.items()}

class pkparser():
    """#TODO save terminus files
    last id full path include dir. assumed XXXpdb.gz as is gotten from RCSB."""
    """it was necessary to use numpy, because I cant make jagged tensors."""
    def __init__(self,gzipped_pdb):
        self.path = gzipped_pdb#"/Users/jessihoernschemeyer/pKaSchNet/pkas.csv" #('/content/drive/MyDrive/pkas.csv') #3directory
        self.pdb = gzipped_pdb[-11:-7]
        self.targets=np.load(f"/home/jrhoernschemeyer/Desktop/data_prep/targets/{self.pdb}.npz")

    def parse_pdb(self):
        pdbspecies, pdbcoors,terminus_lines=[],[],[]
        species,coors,line,lastresi,strii=None,None,None,None,None

        with gzip.open(self.path, "r") as f:
            lines=np.char.array(f.readlines())
            f.close()
        #encode everything if user didnt gzip their filess

        #run
        lines=np.char.array([line for line in lines if np.char.startswith(line,b"ATOM")])#for line in lines if np.char.startswith(line,"ATOM")])
        lines=[line[13:] for line in lines if line[17:20] in fullcode.keys()]
        while lines:
            stri=lines[0]
            if not stri[0:1] == b"H":
                #alt conformations
                if not np.char.isspace(stri[3:4]):
                    strii=lines[1]
                    stri1,stri2=stri[43:47], strii[43:47]
                    if np.char.greater_equal(stri1,stri2):
                        resi=stri[4:14]
                        resinum=resi[5:].strip()
                        if not resinum.isdigit(): #INSERT
                            del lines[0]
                            del lines[0]
                            strii=None
                            continue 

                        if resinum==lastresi:
                            line, lastresi=stri,resinum
                            species.append(elements[list(filter(str.isalpha, line[-4:].decode()))[0]])
                            coors.append((np.float32(line[17:25]),np.float32(line[25:33]),np.float32(line[33:41])))
                            del lines[0]
                            del lines[0]
                        else: #newresi
                            pdbspecies.append(np.array(species))
                            pdbcoors.append(np.array(coors))
                            terminus_lines.append(line)
                            line,lastresi=stri,resi
                            species=[elements[list(filter(str.isalpha, line[-4:].decode()))[0]]]
                            coors=[(np.float32(line[17:25]),np.float32(line[25:33]),np.float32(line[33:41]))]
                            lastresi=resinum
                            del lines[0]
                            del lines[0]
                            strii=None     

                    else:
                            #B is greater
                        resi=strii[4:14]
                        resinum=resi[5:].strip()
                        if not resinum.isdigit(): #INSERT
                            del lines[0]
                            strii=None
                            continue 

                        if resinum==lastresi:
                            line, lastresi=strii,resinum
                            species.append(elements[list(filter(str.isalpha, line[-4:].decode()))[0]])
                            coors.append((np.float32(line[17:25]),np.float32(line[25:33]),np.float32(line[33:41])))
                            del lines[0]
                        else:  #newresi
                            pdbspecies.append(np.array(species))
                            pdbcoors.append(np.array(coors))
                            terminus_lines.append(line)
                            line=strii
                            strii=None
                            species=[elements[list(filter(str.isalpha, line[-4:].decode()))[0]]]
                            coors=[(np.float32(line[17:25]),np.float32(line[25:33]),np.float32(line[33:41]))]
                            lastresi=resinum
                            del lines[0]

                #normal run
                else:
                    resi=stri[4:14]
                    resinum=resi[5:].strip()
                    if not resinum.isdigit(): #INSERTion
                        del lines[0]
                        continue 

                    if resinum==lastresi:
                        line,lastresi=stri,resinum
                        species.append(elements[list(filter(str.isalpha, line[-4:].decode()))[0]])
                        coors.append((np.float32(line[17:25]),np.float32(line[25:33]),np.float32(line[33:41])))
                        del lines[0]
                    else: #newresi
                        pdbspecies.append(np.array(species))
                        pdbcoors.append(np.array(coors))
                        terminus_lines.append(line)
                        line,lastresi=stri,resinum
                        species=[elements[list(filter(str.isalpha, line[-4:].decode()))[0]]]
                        coors=[(np.float32(line[17:25]),np.float32(line[25:33]),np.float32(line[33:41]))]
                        lastresi=resinum
                        del lines[0]
            else: #hydrogens-skiü
                del lines[0]
                continue
                     
        pdbspecies.append(np.array(species))
        pdbcoors.append(np.array(coors))
        terminus_lines.append(line)
        #TODO save terminus lines
        return np.array(pdbspecies[1:],dtype=object), np.array(pdbcoors[1:],dtype=object), [t[:14] for t in terminus_lines[1:]]
    
    def hoods(self,coors,species,cutoff):
        all_coors = np.vstack(coors)
        neigh = NearestNeighbors(algorithm="brute")
        neigh.fit(all_coors)
        idxs,last=[],0
        for z in species:
            idxs.append(len(z) + last - 1)
            last=idxs[-1]
        nbrs = neigh.radius_neighbors(all_coors[idxs],radius=cutoff, return_distance=False)
        all_species=np.concatenate(species)
        return [all_species[n_ixs] for n_ixs in nbrs], [all_coors[n_ixs].astype(np.float32) for n_ixs in nbrs]

    def run(self,cutoff):
        pkpdb=self.targets
        species,coors,ids=self.parse_pdb()
        ids=[np.char.add(np.char.add(np.char.strip(b[-5:]),b[-6:-5]),code[b[4:5]]) for b in ids]
        ids, pidx, midx = np.intersect1d(pkpdb["ids"], ids,return_indices=True)
        species,coors=self.hoods(coors[midx],species[midx],cutoff)
        np.savez_compressed(f"/home/jrhoernschemeyer/Desktop/data_prep/{self.pdb}.npz",z=list(species), pos=coors, pks=pkpdb["pks"][pidx],ids=ids)
        #except:
        #"zip the pdb or find pdb code somehow else lol or seperate home dir from after"
        
        

In [127]:
pkparser(gzipped_pdb=f"/home/jrhoernschemeyer/Desktop/data_prep/structures/pdbs/4b00.pdb.gz").run(cutoff=5)
np.load("/home/jrhoernschemeyer/Desktop/data_prep/4b00.npz",allow_pickle=True)["pks"]

/home/jrhoernschemeyer/.local/lib/python3.6/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


array([ 0.22129 , -0.79569 ,  0.5367  , -1.51569 ,  2.9097  , -0.44776 ,
       -0.55298 ,  0.12613 , -0.87682 , -1.45766 ,  0.831   ,  0.0852  ,
        5.2029  ,  0.6834  , -0.62525 ,  0.13924 , -0.18053 ,  1.1122  ,
        0.9753  , -0.32915 ,  0.119699,  5.4209  , -0.21871 , -0.69426 ,
       -1.59515 , -0.85411 ,  0.2458  , -1.2416  ,  1.9602  , -0.22077 ,
        2.8845  ,  0.627   , -1.39574 ,  1.7719  , -0.1739  ,  0.6228  ,
        0.6435  , -0.78471 , -0.1947  ,  0.1592  , -0.1032  , -0.0735  ,
       -0.29553 ,  0.22327 ,  1.6525  , -0.86716 ,  3.8939  , -1.3183  ,
       -0.35249 ,  0.58418 ,  0.11886 ,  1.7884  , -0.4526  ,  1.03342 ,
        0.17446 ,  2.7562  , -1.88992 ,  0.4164  , -0.23051 , -0.70252 ,
       -2.03285 , -0.20612 , -0.7963  ,  0.03165 ,  1.63543 , -1.09534 ,
        1.2517  , -3.33058 ,  0.16046 ,  0.33497 ,  0.33912 ,  5.4129  ,
        2.2805  , -0.49152 ,  0.3632  ,  0.4697  ,  1.6012  ,  0.8952  ,
        0.22312 ,  0.16771 , -1.32095 ,  0.70319 , 